# Importing All the necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from termcolor import colored 

import warnings
warnings.filterwarnings("ignore")

# Loading the DataSet

In [ ]:
df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")
df.head()


In [ ]:
print(colored("There are {} rows and {} Colimns in dataset".format(df.shape[0], df.shape[1]), attrs=['bold']))

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.isnull())
plt.show()

for i in df.columns:
    null_rate = df[i].isna().sum()/len(df) * 100
    
    if null_rate > 0:
        print("{}'s null rate: {}%".format(i, round(null_rate, 2)))


In [ ]:
df.drop(['director'], axis=1, inplace=True)
df.head()

In [ ]:
df['country'].replace(np.nan, 'United States', inplace = True)
df['cast'].replace(np.nan, 'No Data', inplace = True)

print(colored("I will drop the missing rows from the columns 'date_added' and 'rating' since these have only {} missing rows in total." .format(df.isnull().sum().sum()), attrs=['bold'] ))

In [ ]:
df.dropna(inplace = True)
print(colored("There are {} and {} columns.".format(df.shape[0], df.shape[1]), attrs=['bold']))

# Checking the data thoroughly

In [ ]:
df.info()

# date_added is of type object, Converting it into of type datetime format.

In [ ]:
df["date_added"] = pd.to_datetime(df['date_added'])

df['moth_added'] = df['date_added'].dt.month
df['month_name_added'] = df['date_added'].dt.month_name()
df['year_added'] = df['date_added'].dt.year

df.drop('date_added', axis=1, inplace=True)
df.head()

# Looking at the CONTENT TYPE

In [ ]:
plt.figure(figsize=(10,5))
plt.pie(df['type'].value_counts().sort_values(), labels=df['type'].value_counts().index, explode=[0.05, 0], autopct='%1.2f%%',colors=['Green','grey'])
plt.show()

In [ ]:
from collections import Counter
country_data = df['country']
country_count = pd.Series(dict(Counter(','.join(country_data).replace(',', ',').replace(',',',').split(',')))).sort_values(ascending=False)

In [ ]:
top20country = country_count.head(20)

In [ ]:
from matplotlib import gridspec

fig = plt.figure(figsize=(20, 7))

gs = gridspec.GridSpec(nrows=1, ncols=2, height_ratios=[6], width_ratios=[10, 5])

ax = plt.subplot(gs[0])
sns.barplot(top20country.index, top20country, ax=ax, palette="RdGy")
ax.set_xticklabels(top20country.index, rotation='90')
ax.set_title('Top 20 countries with most contents', fontsize=15, fontweight='bold')

ax2 = plt.subplot(gs[1])
ax2.pie(top20country, labels=top20country.index, shadow=True, startangle=0, colors=sns.color_palette("RdGy",n_colors=20), autopct='%1.2f%%')
ax2.axis('equal')
plt.show()

In [ ]:
df_tv = df[df["type"] == "Tv Show"]
df_movies = df[df["type"] == "Movie"]

In [ ]:
df_content = df['year_added'].value_counts().reset_index().rename(columns = 
   {
       'year_added' : 'count', 'index' : 'year_added'}).sort_values('year_added') 

df_content['percent'] = df_content['count'].apply(lambda x : 100*x/sum(df_content['count']))

df_tv1 = df_tv['year_added'].value_counts().reset_index().rename(columns = {
    'year_added' : 'count', 'index' : 'year_added'}).sort_values('year_added')
df_tv1['percent'] = df_tv1['count'].apply(lambda x : 100*x/sum(df_tv1['count']))

df_movies1 = df_movies['year_added'].value_counts().reset_index().rename(columns = 

{
    'year_added' : 'count', 'index' : 'year_added'}).sort_values('year_added')

df_movies1['percent'] = df_movies1['count'].apply(lambda x : 100*x/sum(df_movies1['count']))


t1 = go.Scatter(x=df_movies1['year_added'], y=df_movies1["count"], name="Movies", marker=dict(color="#a678de"))
t2 = go.Scatter(x=df_tv1['year_added'], y=df_tv1["count"], name="Tv Shows", marker=dict(color="#6ad49b"))
t3 = go.Scatter(x=df_content['year_added'], y=df_content["count"], name="Total Contents", marker=dict(color="brown"))

data = [t1, t2, t3]

layout = go.Layout(title = "Content Added Over the Years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()
 

# Contents Over The Month

In [ ]:
df_content = df[['moth_added','month_name_added']].value_counts().reset_index().rename(columns = {
    0 : 'count'}).sort_values('moth_added').drop('moth_added',axis=1)
df_content['percent'] = df_content['count'].apply(lambda x : 100*x/sum(df_content['count']))


df_tv2 = df_tv[['moth_added','month_name_added']].value_counts().reset_index().rename(columns = {
    0 : 'count'}).sort_values('moth_added').drop('moth_added',axis=1)
df_tv2['percent'] = df_tv2['count'].apply(lambda x : 100*x/sum(df_tv2['count']))


df_movies2 = df_movies[['moth_added','month_name_added']].value_counts().reset_index().rename(columns = {
    0 : 'count'}).sort_values('moth_added').drop('moth_added',axis=1)
df_movies2['percent'] = df_movies2['count'].apply(lambda x : 100*x/sum(df_movies2['count']))

t1 = go.Scatter(x=df_movies2['month_name_added'], y=df_movies2["count"], name="Movies", marker=dict(color="#a678de"))
t2 = go.Scatter(x=df_tv2['month_name_added'], y=df_tv2["count"], name="TV Shows", marker=dict(color="#6ad49b"))
t3 = go.Scatter(x=df_content['month_name_added'], y=df_content["count"], name="Total Contents", marker=dict(color="Brown"))


data = [t1, t2, t3]

layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()



# A look to different Genres in dataset

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def relation_heatmap(df, title):
    df['genre'] = df['listed_in'].apply(lambda x : x.replace(',',',').replace(',',',').split(','))
    Types = []
    for i in df['genre']: Types += i
    Types=set(Types)
    print("There are {} types in the Netflix {} Dataset".format(len(Types), title))
    test = df['genre']
    mlb = MultiLabelBinarizer()
    res = pd.DataFrame(mlb.fit_transform(test), columns=mlb.classes_, index=test.index)
    corr = res.corr()
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    fig, ax = plt.subplots(figsize=(10, 7))
    pl = sns.heatmap(corr, mask=mask, cmap="coolwarm", vmax=.5, vmin=-.5, center=0, square=True, linewidths=.7,cbar_kws={"shrink":0.6})
    plt.show()
        

In [ ]:
relation_heatmap(df_movies, 'Movie')

In [ ]:
relation_heatmap(df_tv, 'Tv Show')

In [ ]:
rating_order_movie =  ['G', 'TV-Y', 'TV-G', 'PG', 'TV-Y7', 'TV-Y7-FV', 'TV-PG', 'PG-13', 'TV-14', 'R', 'NC-17', 'TV-MA']
rating_order_tv =  ['G', 'TV-Y', 'TV-G', 'TV-Y7', 'TV-Y7-FV', 'TV-PG', 'TV-14', 'R', 'TV-MA']

In [ ]:
movie_rating = df_movies['rating'].value_counts()[rating_order_movie]
tv_rating = df_tv['rating'].value_counts()[rating_order_tv].fillna(0)

In [ ]:
def rating_barplot(data, title, height, h_lim=None):
    fig, ax = plt.subplots(1,1, figsize=(15, 7))
    if h_lim :
        ax.set_ylim(0, h_lim)
    ax.bar(data.index, data,  color="#d0d0d0", width=0.6, edgecolor='black')

    color =  ['green',  'blue',  'orange',  'red']
    span_range = [[0, 2], [3,  6], [7, 8], [9, 11]]

    for idx, sub_title in enumerate(['Little Kids', 'Older Kids', 'Teens', 'Mature']):
        ax.annotate(sub_title,
                    xy=(sum(span_range[idx])/2 ,height),
                    xytext=(0,0), textcoords='offset points',
                    va="center", ha="center",
                    color="w", fontsize=16, fontweight='bold',
                    bbox=dict(boxstyle='round4', pad=0.4, color=color[idx], alpha=0.6))
        ax.axvspan(span_range[idx][0]-0.4,span_range[idx][1]+0.4,  color=color[idx], alpha=0.1)

    ax.set_title(f'Distribution of {title} Rating', fontsize=20, fontweight='bold', position=(0.5, 1.0+0.03))
    plt.show()

In [ ]:
rating_barplot(movie_rating,'Movie', 1200)

In [ ]:
rating_barplot(tv_rating,'TV Show' , 700, 800)

# Movie Duration

In [ ]:
from scipy.stats import norm

plt.figure(figsize=(15, 7))
sns.distplot(df_movies['duration'].str.extract('(\d+)'), fit=norm, kde=False, color=['red'])
plt.title('Distplot with normal distribution of movies', fontweight="bold")
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
ax = sns.barplot(df_tv['duration'], order = df_tv['duration'].value_counts().index, palette="RdGy")
plt.title('Countplot for season in TV Shows', fontweight="bold")
plt.xticks(rotation=90)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, (p.get_height() * 1.005)))
    

plt.figure(figsize=(15, 7))
ax = sns.barplot(x=((df_tv['duration'].value_counts()/df_tv.shape[0])*100).index,
                 y=round(((df_tv['duration'].value_counts()/df_tv.shape[0])*100),2).values,
                 palette="RdGy")
plt.title('Rectangle of Seasons in TV show', fontweight="bold")
plt.xticks(rotation=90)
for p in ax.patches:
     ax.annotate(str(p.get_height()), (p.get_x() * 1.005, (p.get_height() * 1.005)))
        
plt.show()        

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = df_movies["listed_in"].value_counts().head(10).index,
            y = df_movies["listed_in"].value_counts().head(10).values,palette="RdGy")
plt.xticks(rotation=80)
plt.title("Top10 Genre in Movies",fontweight="bold")
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = df_tv["listed_in"].value_counts().head(10).index,
            y = df_tv["listed_in"].value_counts().head(10).values,palette="RdGy")
plt.xticks(rotation=80)
plt.title("Top10 Genre in TV Shows",fontweight="bold")
plt.show()

In [ ]:
df['cast_name'] = df['cast'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 
cast_count = []
for i in df['cast_name']: cast_count += i
    
cast_dict = dict((i, cast_count.count(i)) for i in cast_count)

df_cast_count = pd.DataFrame(cast_dict.values(),cast_dict.keys()).reset_index().sort_values(0,ascending=False).rename(
    columns = {'index' : 'cast_name', 0 : 'count'}).iloc[1:21]

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x='cast_name',y='count',data=df_cast_count,palette="RdGy")
plt.title("Top20 Artist on Netflix",fontweight="bold")
plt.xticks(rotation=90)
plt.show()